In [60]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os
import seaborn as sns
import sys
import os
from IPython.display import display
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')


In [61]:
emp_data = pd.read_excel('C:/Users/GANESHA/Downloads/project/Data/INX_Future_Inc_Employee_Performance_CDS_Project2_Data_V1.8.xls')

In [62]:
emp_data['Department_Role'] = emp_data['EmpJobRole'] + " - " + emp_data['EmpDepartment']
emp_data_columns = emp_data.columns.values
X = emp_data[emp_data_columns[0:29]]
Y = emp_data['PerformanceRating']
X.drop('PerformanceRating', axis=1, inplace=True)
X.drop('EmpNumber', axis=1, inplace = True)

In [63]:
SEED = 6
np.random.seed(SEED)

In [64]:
X_Gender = pd.get_dummies(X['Gender'], drop_first = False , sparse = True)
X_Education = pd.get_dummies(X['EducationBackground'], drop_first = False , sparse = True)
X_MaritalStatus = pd.get_dummies(X['MaritalStatus'], drop_first = False , sparse = True)
X_EmpDepartment = pd.get_dummies(X['EmpDepartment'], drop_first = False , sparse = True)
X_EmpJobRole = pd.get_dummies(X['EmpJobRole'], drop_first = False , sparse = True)
X_BusinessTravelFrequency = pd.get_dummies(X['BusinessTravelFrequency'], drop_first = False , sparse = True)
X_OverTime = pd.get_dummies(X['OverTime'], drop_first = False, sparse = True)
X_Department_Role = pd.get_dummies(X['Department_Role'], drop_first = False , sparse = True)
X_Attrition = pd.get_dummies(X['Attrition'], drop_first = False, sparse = True)

In [65]:
X['Gender'] = X_Gender
X['EducationBackground'] = X_Education
X['MaritalStatus'] = X_MaritalStatus
X['EmpDepartment'] = X_EmpDepartment
X['EmpJobRole'] = X_EmpJobRole
X['BusinessTravelFrequency'] = X_BusinessTravelFrequency
X['OverTime'] = X_OverTime
X['Department_Role'] = X_Department_Role
X['Attrition'] = X_Attrition
X.shape

(1200, 27)

Spliting data into Xtran, Xtest , Y train and Y test

In [66]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state = 0, stratify = emp_data['PerformanceRating'])

In [67]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform

## creating RandomSearch for tunning parameters

In [68]:
class RandomSearch(object):
    
    def __init__(self,X_train,y_train,model,hyperparameters):
        
        self.X_train = X_train
        self.y_train = y_train
        self.model = model
        self.hyperparameters = hyperparameters
        
    def RandomSearch(self):
        # Create randomized search 10-fold cross validation and 100 iterations
        cv = 10
        clf = RandomizedSearchCV(self.model,
                                 self.hyperparameters,
                                 random_state=1,
                                 n_iter=100,
                                 cv=cv,
                                 verbose=0,
                                 n_jobs=-1,
                                 )
        # Fit randomized search
        best_model = clf.fit(self.X_train, self.y_train)
        message = (best_model.best_score_, best_model.best_params_)
        print("Best: %f using %s" % (message))

        return best_model,best_model.best_params_
    
    def BestModelPridict(self,X_test):
        
        best_model,_ = self.RandomSearch()
        pred = best_model.predict(X_test)
        return pred

## defining GridSearch for hyper tunning parameters

In [69]:
class GridSearch(object):
    
    def __init__(self,X_train,y_train,model,hyperparameters):
        
        self.X_train = X_train
        self.y_train = y_train
        self.model = model
        self.hyperparameters = hyperparameters
        
    def GridSearch(self):
        # Create randomized search 10-fold cross validation and 100 iterations
        cv = 10
        clf = GridSearchCV(self.model,
                                 self.hyperparameters,
                                 cv=cv,
                                 verbose=0,
                                 n_jobs=-1,
                                 )
        # Fit randomized search
        best_model = clf.fit(self.X_train, self.y_train)
        message = (best_model.best_score_, best_model.best_params_)
        print("Best: %f using %s" % (message))

        return best_model,best_model.best_params_
    
    def BestModelPridict(self,X_test):
        
        best_model,_ = self.GridSearch()
        pred = best_model.predict(X_test)
        return pred

In [70]:
def floatingDecimals(f_val, dec=3):
        prc = "{:."+str(dec)+"f}" #first cast decimal as str
    #     print(prc) #str format output is {:.3f}
        return float(prc.format(f_val))

## Using Logistic Regression as ml model

In [71]:
model = LogisticRegression()
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter distribution using uniform distribution
C = uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)
LR_RandSearch = RandomSearch(X_train,Y_train,model,hyperparameters)
# LR_best_model,LR_best_params = LR_RandSearch.RandomSearch()
Prediction_LR = LR_RandSearch.BestModelPridict(X_test)
print('prediction on test set is:' ,floatingDecimals((Y_test == Prediction_LR).mean(),7))

Best: 0.770238 using {'C': 2.7460037107263346, 'penalty': 'l1'}
prediction on test set is: 0.7666667


## using KNN as ML Model

In [72]:
model_KNN = KNeighborsClassifier()
neighbors = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
param_grid = dict(n_neighbors=neighbors)

In [73]:
KNN_GridSearch = GridSearch(X_train,Y_train,model_KNN,param_grid)
Prediction_KNN = KNN_GridSearch.BestModelPridict(X_test)
print('prediction on test set is:' ,floatingDecimals((Y_test == Prediction_KNN).mean(),7))

Best: 0.733333 using {'n_neighbors': 12}
prediction on test set is: 0.7333333


## Using Decission Tree Classifier as ML Model

In [74]:
from scipy.stats import randint
max_depth_value = [3, None]
max_features_value =  randint(1, 4)
min_samples_leaf_value = randint(1, 4)
criterion_value = ["gini", "entropy"]

In [75]:
param_grid = dict(max_depth = max_depth_value,
                  max_features = max_features_value,
                  min_samples_leaf = min_samples_leaf_value,
                  criterion = criterion_value)

In [76]:
model_CART = DecisionTreeClassifier()
CART_RandSearch = RandomSearch(X_train,Y_train,model_CART,param_grid)
Prediction_CART = CART_RandSearch.BestModelPridict(X_test)
print('prediction on test set is:' ,floatingDecimals((Y_test == Prediction_CART).mean(),7))

Best: 0.803571 using {'criterion': 'entropy', 'max_depth': None, 'max_features': 3, 'min_samples_leaf': 3}
prediction on test set is: 0.7166667


## Using SVM as ML Model

In [77]:
c_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.3, 1.5, 1.7, 2.0]
kernel_values = [ 'linear' , 'poly' , 'rbf' , 'sigmoid' ]
param_grid = dict(C=c_values, kernel=kernel_values)
model_SVC = SVC()

In [78]:
SVC_GridSearch = GridSearch(X_train,Y_train,model_SVC,param_grid)
Prediction_SVC = SVC_GridSearch.BestModelPridict(X_test)
print('prediction on test set is:' ,floatingDecimals((Y_test == Prediction_SVC).mean(),7))

Best: 0.784524 using {'C': 0.5, 'kernel': 'linear'}
prediction on test set is: 0.7888889


## Using AdaBoost classifier as ML Model

In [79]:
learning_rate_value = [.01,.05,.1,.5,1]
n_estimators_value = [50,100,150,200,250,300,500]

param_grid = dict(learning_rate=learning_rate_value, n_estimators=n_estimators_value)

In [80]:
model_Ad = AdaBoostClassifier()
Ad_GridSearch = GridSearch(X_train,Y_train,model_Ad,param_grid)
Prediction_Ad = Ad_GridSearch.BestModelPridict(X_test)
print('prediction on test set is:' ,floatingDecimals((Y_test == Prediction_Ad).mean(),7))

Best: 0.845238 using {'learning_rate': 0.05, 'n_estimators': 250}
prediction on test set is: 0.8361111


## Using GradientBoostng Algorithm as ML Model

In [81]:
learning_rate_value = [.01,.05,.1,.5,1]
n_estimators_value = [50,100,150,200,250,300,400,500,1000]

param_grid = dict(learning_rate=learning_rate_value, n_estimators=n_estimators_value)

In [82]:
model_GB = GradientBoostingClassifier()
GB_GridSearch = GridSearch(X_train,Y_train,model_GB,param_grid)
Prediction_GB = GB_GridSearch.BestModelPridict(X_test)
print('prediction on test set is:' ,floatingDecimals((Y_test == Prediction_GB).mean(),7))

Best: 0.909524 using {'learning_rate': 0.01, 'n_estimators': 150}
prediction on test set is: 0.8944444


# Combining all models with tunned hyper parameters 

In [83]:
from sklearn.ensemble import VotingClassifier

In [84]:
param = {'C': 3.5055566091841532, 'penalty': 'l1'}
model1 = LogisticRegression(**param)

param = {'n_neighbors': 12}
model2 = KNeighborsClassifier(**param)

param = {'C': 0.5, 'kernel': 'linear'}
model3 = SVC(**param)

param = {'criterion': 'entropy', 'max_depth': 3, 'max_features': 2, 'min_samples_leaf': 3}
model4 = DecisionTreeClassifier(**param)

param = {'learning_rate': 0.05, 'n_estimators': 250}
model5 = AdaBoostClassifier(**param)

param = {'learning_rate': 0.01, 'n_estimators': 150}
model6 = GradientBoostingClassifier(**param)

model7 = GaussianNB()

model8 = RandomForestClassifier()

model9 = ExtraTreesClassifier()

In [85]:
estimators = [('LR',model1), ('KNN',model2), ('SVC',model3),
              ('DT',model4), ('ADa',model5), ('GB',model6),
              ('NB',model7), ('RF',model8),  ('ET',model9)]

In [86]:
kfold = StratifiedKFold(n_splits=10, random_state=SEED)
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, X_train,Y_train, cv=kfold)
print('Accuracy on train: ',results.mean())
ensemble_model = ensemble.fit(X_train,Y_train)
pred = ensemble_model.predict(X_test)
print('Accuracy on test:' , (Y_test == pred).mean())

Accuracy on train:  0.8322275814220335
Accuracy on test: 0.8166666666666667
